In [ ]:
# VADER Sentiment Analysis

import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
Japan = pd.read_csv('Preprocessed_Headline_Korea.csv')

analyser = SentimentIntensityAnalyzer()


def print_sentiment_scores(row):
    senti = row['Headline_Original']
    japan_senti = analyser.polarity_scores(senti)
    return japan_senti


Japan['Senti_Original'] = Japan.apply(print_sentiment_scores, axis=1)

Japan.to_csv('Korea_Headline_Original_VADER.csv', index=False)
print(Japan)

In [ ]:
# Textblob Sentiment Analysis

import pandas as pd
from textblob import TextBlob

Japan = pd.read_csv('Preprocessed_Headline_Korea.csv')


def blob_senti(Headline_Original):
    return TextBlob(Headline_Original).sentiment.polarity


Japan['Senti_Original'] = Japan.Headline_Original.apply(blob_senti)


def blob_obj(Headline_Original):
    return TextBlob(Headline_Original).sentiment.subjectivity


Japan['Subjectivity_Original'] = Japan.Headline_Original.apply(blob_obj)

Japan.to_csv('Korea_Headline_Original_Textblob.csv', index=False)
print(Japan)

In [ ]:
# SentiWordNet Sentiment Analysis 

import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet as wn
from nltk.corpus import sentiwordnet as swn
from nltk import sent_tokenize, word_tokenize, pos_tag
Japan = pd.read_csv('Preprocessed_Headline_India.csv')
lemmatizer = WordNetLemmatizer()


def penn_to_wn(tag):
    """
    Convert between the PennTreebank tags to simple Wordnet tags
    """
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None


def swn_polarity(row):
    """
    Return a sentiment polarity: 0 = negative, 1 = positive
    """

    sentiment = 0.0
    tokens_count = 0
    text = row['Headline_Original']

    raw_sentences = sent_tokenize(text)
    for raw_sentence in raw_sentences:
        tagged_sentence = pos_tag(word_tokenize(raw_sentence))

        for word, tag in tagged_sentence:
            wn_tag = penn_to_wn(tag)
            if wn_tag not in (wn.NOUN, wn.ADJ, wn.ADV):
                continue

            lemma = lemmatizer.lemmatize(word, pos=wn_tag)
            if not lemma:
                continue

            synsets = wn.synsets(lemma, pos=wn_tag)
            if not synsets:
                continue

            # Take the first sense, the most common
            synset = synsets[0]
            swn_synset = swn.senti_synset(synset.name())

            sentiment += swn_synset.pos_score() - swn_synset.neg_score()
            tokens_count += 1

    # judgment call ? Default to positive or negative
    if not tokens_count:
        return 0

    # sum greater than 0 => positive sentiment
    if sentiment >= 0:
        return 1

    # negative sentiment
    return 0


Japan['Senti_Original'] = Japan.apply(swn_polarity, axis=1)
Japan.to_csv('India_Headline_Original_SentiWordNet.csv', index=False)


print(Japan)